In [1]:
# !pip install ipywidgets
# from ipywidgets import FloatProgressac

  Using cached fqdn-1.5.1-py3-none-any.whl.metadata (1.4 kB)
  Using cached isoduration-20.11.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached uri_template-1.3.0-py3-none-any.whl.metadata (8.8 kB)
Using cached fqdn-1.5.1-py3-none-any.whl (9.1 kB)
Using cached isoduration-20.11.0-py3-none-any.whl (11 kB)
Using cached uri_template-1.3.0-py3-none-any.whl (11 kB)


In [2]:
!pip install tqdm
!pip install matplotlib
!pip install scikit-learn

In [3]:
# PyTorch 관련 라이브러리
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms


In [4]:
# 이미지 처리 및 컴퓨터 비전 관련 라이브러리
import os
import cv2
from tqdm import tqdm_notebook as tqdm
import random
from matplotlib import pyplot as plt
import time
from PIL import Image, ImageFile
import numpy as np
from sklearn.model_selection import train_test_split


In [5]:
# 사전 학습된 모델 관련 라이브러리
from torchvision import models
from torchvision.models import ResNet18_Weights


In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
#pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [7]:
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
batch_size = 32

In [8]:
class ImageTransform():
    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
                transforms.RandomResizedCrop(resize, scale=(0.5, 1.0)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ]),
            'val': transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ])
        }

    def __call__(self, img, phase):
        return self.data_transform[phase](img)

In [9]:
acc_directory = r'C:/Users/SKTPJ13/Desktop/Data0816/acc'
bk_directory = r'C:/Users/SKTPJ13/Desktop/Data0816/brake'
acc_push_directory = r"C:/Users/SKTPJ13/Desktop/Data0816/acc_push"
bk_push_directory = r"C:/Users/SKTPJ13/Desktop/Data0816/brake_push"

acc_directory = sorted([os.path.join(acc_directory, f) for f in os.listdir(acc_directory)])
bk_directory = sorted([os.path.join(bk_directory, f) for f in os.listdir(bk_directory)])
acc_push_directory = sorted([os.path.join(acc_push_directory, f) for f in os.listdir(acc_push_directory)])
bk_push_directory = sorted([os.path.join(bk_push_directory, f) for f in os.listdir(bk_push_directory)])

images_filepaths = [*acc_directory, *bk_directory, *acc_push_directory, *bk_push_directory]

# tqdm으로 진행 상황 표시
correct_images_filepaths = [i for i in tqdm(images_filepaths) if cv2.imread(i) is not None]

random.seed(42)
random.shuffle(correct_images_filepaths)

print(len(correct_images_filepaths))


C:\Users\SKTPJ13\AppData\Local\Temp\ipykernel_17836\3358847827.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  correct_images_filepaths = [i for i in tqdm(images_filepaths) if cv2.imread(i) is not None]


  0%|          | 0/20096 [00:00<?, ?it/s]

20096


In [10]:


# 손상된 이미지도 로드할 수 있도록 설정
#ImageFile.LOAD_TRUNCATED_IMAGES = True

def load_images_in_batches(image_paths, batch_size=10):
    correct_images = []
    for i in range(0, len(image_paths), batch_size):
        batch = image_paths[i:i + batch_size]
        for path in batch:
            try:
                # PIL을 사용해 이미지 열기 (손상된 이미지를 걸러내기 위함)
                img = Image.open(path)
                img.verify()  # 이미지를 검증하여 손상 여부 확인
                image = cv2.imread(path)  # OpenCV로 이미지 읽기
                if image is not None:
                    correct_images.append(path)
            except (IOError, SyntaxError, cv2.error) as e:
                print(f"Skipping corrupted image: {path}")
    return correct_images

# images_filepaths는 이미지 파일 경로들의 리스트
correct_images_filepaths = load_images_in_batches(images_filepaths)


In [11]:

# 먼저, 학습 세트와 나머지 데이터를 분할
train_images_filepaths, temp_paths = train_test_split(correct_images_filepaths, test_size=0.4, random_state=42)

# 나머지 데이터(검증 + 테스트)를 검증 세트와 테스트 세트로 분할
val_images_filepaths, test_images_filepaths = train_test_split(temp_paths, test_size=0.5, random_state=42)


In [12]:

class CarDataset(Dataset):
    def __init__(self, file_list, transform=None, phase='train'):
        self.file_list = file_list
        self.transform = transform
        self.phase = phase

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path)
        img_transformed = self.transform(img, self.phase)

        label = img_path.split('/')[-1].split('\\')[0]
        #print(label)
        if 'brake_push' in label:
            label = 1
        elif 'acc_push' in label:
            label = 0
        elif 'acc' in label:
            label = 2
        elif 'brake' in label:
            label = 3

        return img_transformed, label

In [15]:

train_dataset = CarDataset(train_images_filepaths, transform=ImageTransform(size, mean, std), phase='train')
val_dataset = CarDataset(val_images_filepaths, transform=ImageTransform(size, mean, std), phase='val')

index = 0
print(train_dataset.__getitem__(index)[0].size()) #이미지의 크기
print(train_dataset.__getitem__(index)[1]) #첫 번째 이미지의 레이블

torch.Size([3, 224, 224])
0


In [16]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
dataloader_dict = {'train': train_dataloader, 'val': val_dataloader}

batch_iterator = iter(train_dataloader)
inputs, label = next(batch_iterator)


In [17]:

# Pretrained ResNet18 모델 불러오기

model = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)  # 4는 클래스 수에 따라 조정


In [18]:
optimizer = optim.Adam(model.parameters(), lr=1e-7)
criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [19]:
def train_model(model, dataloader_dict, criterion, optimizer, num_epoch):
    since = time.time()
    best_acc = 0.0

    for epoch in range(num_epoch):
        print('Epoch {}/{}'.format(epoch + 1, num_epoch))
        print('-'*20)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            epoch_loss = 0.0
            epoch_corrects = 0

            for inputs, labels in tqdm(dataloader_dict[phase]):

                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    pred_prob, preds = torch.max(outputs, 1) # 예측확률, 예측정답
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    epoch_loss += loss.item() * inputs.size(0)
                    epoch_corrects += torch.sum(preds == labels.data)

            epoch_loss = epoch_loss / len(dataloader_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloader_dict[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    return model,best_model_wts

In [ ]:
import time
ImageFile.LOAD_TRUNCATED_IMAGES = True
num_epoch = 50
model = model.to(device)
model,best_model_wts = train_model(model, dataloader_dict, criterion, optimizer, num_epoch)

Epoch 1/50
--------------------


C:\Users\SKTPJ13\AppData\Local\Temp\ipykernel_17836\3197702065.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs, labels in tqdm(dataloader_dict[phase]):


  0%|          | 0/377 [00:00<?, ?it/s]

train Loss: 1.4559 Acc: 0.2939


  0%|          | 0/126 [00:00<?, ?it/s]

...

  0%|          | 0/377 [00:00<?, ?it/s]

train Loss: 0.8493 Acc: 0.7425


  0%|          | 0/126 [00:00<?, ?it/s]

val Loss: 0.8123 Acc: 0.7295
Epoch 17/50
--------------------


  0%|          | 0/377 [00:00<?, ?it/s]

In [ ]:
def test(model, test_images_filepaths):
    id_list = []
    pred_list = []
    _id=0
    with torch.no_grad():
        for test_path in tqdm(test_images_filepaths):
            img = Image.open(test_path)
            _id =test_path.split('/')[-1]
            transform = ImageTransform(size, mean, std)
            img = transform(img, phase='val') #val로 설정하면 랜덤 주지 않음
            img = img.unsqueeze(0) #배치가 1인것을 추가해줌 (토치는 패치단위로 학습하도록 되어있으므로 첫번째에 무조건 배치사이즈가 들어가야함)
            #unsqueeze는 해당자리에 1을 넣어줌 현재는 0에 넣었으므로 맨 앞에 배치사이즈를 추가함
            img = img.to(device)

            model.eval()
            outputs = model(img)
            preds, pred_ans = torch.max(outputs, 1)
            id_list.append(_id)
            pred_list.append(pred_ans.cpu().item())
    return id_list, pred_list



In [ ]:
# 모델 테스트
model = model.to(device)
id_list, pred_list= test(model, test_images_filepaths)

In [ ]:
true_labels=[]
for i in id_list:
    if 'brake_push' in i:
      true_labels.append(1)
    elif 'acc_push' in i:
      true_labels.append(0)
    elif 'acc' in i:
      true_labels.append(2)
    elif 'brake' in i:
      true_labels.append(3)


In [ ]:
def calculate_accuracy(true_labels, pred_labels):
    correct = sum(t == p for t, p in zip(true_labels, pred_labels))
    accuracy = correct / len(true_labels)
    return accuracy

In [ ]:
def visualize_predictions(image_paths, true_labels, pred_labels, num_images=5):
    plt.figure(figsize=(10, 10))
    for i in range(num_images):
        img = Image.open(image_paths[i])
        true_label = true_labels[i]
        pred_label = pred_labels[i]

        plt.subplot(1, num_images, i + 1)
        plt.imshow(img)
        plt.title(f'True: {true_label}\nPred: {pred_label}')
        plt.axis('off')

    plt.show()

In [ ]:
# 정확도 계산
accuracy = calculate_accuracy(true_labels, pred_list)
print(f'Accuracy: {accuracy:.4f}')

# 예측 시각화
visualize_predictions(test_images_filepaths, true_labels, pred_list)


In [ ]:
torch.save(best_model_wts, 'data_0815_left_best_model_wts_18_epoch50.pth')
